# TSA Chapter 5: GARCH Forecasting

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_forecast/TSA_ch5_forecast.ipynb)

GARCH volatility forecasting with rolling window approach.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# GARCH forecast vs realized volatility
returns = btc_returns
n_train = int(len(returns) * 0.8)
test = returns.iloc[n_train:]

forecasts = []
for i in range(min(len(test), 200)):
    window = returns.iloc[:n_train + i]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    forecasts.append(np.sqrt(fcast.variance.values[-1, 0]))
    if (i+1) % 50 == 0: print(f"  {i+1}/{min(len(test), 200)}")

test_sub = test.iloc[:len(forecasts)]
realized = np.abs(test_sub.values)
realized_smooth = pd.Series(realized, index=test_sub.index).rolling(20).mean()

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(test_sub.index, forecasts, color=BLUE, lw=0.8, label='GARCH(1,1) $\\hat{\\sigma}_t$')
ax.plot(test_sub.index, realized_smooth, color=RED, lw=0.8, alpha=0.7, label='$|r_t|$ (20-day MA)')
ax.set_ylabel('Volatility (%)')
ax.set_title('GARCH(1,1) Forecast vs Realised Volatility (Bitcoin)', fontweight='bold')
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(ax.get_xticklabels(), rotation=30, ha='right')
legend_bottom(ax, ncol=2, y=-0.25)
fig.tight_layout(); fig.subplots_adjust(bottom=0.25)
save_chart(fig, 'garch_forecast'); plt.show()